In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [9]:
vine_df = df.selectExpr(["review_id", "cast(star_rating as int) star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [10]:
# Create dataframe for total_votes >= 20
total_votes_df = vine_df.filter(vine_df["total_votes"] >= 20)
total_votes_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [11]:
# Create dataframe where helpful_votes / total_votes >= 50%
helpful_votes_df = total_votes_df.filter((total_votes_df["helpful_votes"] / total_votes_df["total_votes"]) >= .50)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

In [12]:
# Create dataframe that shows all rows where a review was part of the Vine program
paid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "Y")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15PXA9XU9ZKSU|          4|           29|         29|   Y|                N|
|R2LJ4QYQ4PYDEH|          5|           24|         26|   Y|                N|
|R2PJWWQZ37WCIC|          5|           95|         96|   Y|                N|
|R33G2VDKRN2USY|          5|           21|         21|   Y|                N|
|R13R4X37T2U50A|          4|           52|         57|   Y|                N|
|R1X1J2NAQ39DF6|          4|           29|         34|   Y|                N|
|R2UV5XHFNMPXOX|          5|           60|         61|   Y|                N|
|R31V08BCC9X3MF|          5|           18|         22|   Y|                N|
|R2G00QWTYW9N2G|          5|           86|         91|   Y|                N|
| RL8H3NKKP989J|          5|           30|         34|   Y|     

In [13]:
# Create dataframe that shows all rows where a review was not part of the Vine program
unpaid_df = helpful_votes_df.filter(helpful_votes_df["vine"] == "N")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R16YC6SMGKA8SR|          5|           23|         23|   N|                Y|
| R5O4WE9UM60B0|          1|           29|         29|   N|                Y|
|R2EKRVCRC7U0IY|          1|           22|         24|   N|                Y|
|R2OKV47GETH0L7|          5|           24|         24|   N|                Y|
| R36LII9IITE17|          1|           22|         24|   N|                Y|
|R35VKNE16PFY0H|          5|           22|         24|   N|                N|
|R3TG008LHHZZIN|          5|          123|        125|   N|                Y|
|R2FYQBKCC1XG4M|          5|           20|         20|   N|                Y|
| R63PV336NI5X7|          1|           27|         28|   N|                Y|
|R1R42WPHB5ZSWI|          5|           51|         52|   N|     

**PAID ANALYSIS**

In [38]:
# Total number of reviews (paid)
# from pyspark.sql.functions import sum
from pyspark.sql.functions import count
total_paid_df = paid_df.count()
total_paid_df


21

In [23]:
# Total number of 5 star reviews (paid)
five_paid_df = paid_df.filter(paid_df["star_rating"] == 5).count()
five_paid_df

11

In [43]:
# Percentage of 5 star reviews (paid)
per_five_paid_df = (five_paid_df / total_paid_df) * 100
per_five_paid_df

52.38095238095239

**UNPAID ANALYSIS**

In [40]:
# Total number of reviews (unpaid)
total_unpaid_df = unpaid_df.count()
total_unpaid_df

7689

In [25]:
# Total number of 5 star reviews (unpaid)
five_unpaid_df = unpaid_df.filter(unpaid_df["star_rating"] == 5).count()
five_unpaid_df

4444

In [42]:
# Percentage of 5 star reviews (unpaid)
per_five_unpaid_df = (five_unpaid_df / total_unpaid_df) *100
per_five_unpaid_df

57.79685264663805